In [ ]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip /content/fcis-sc-deeplearning-competition.zip

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!unzip  '/content/drive/My Drive/test.zip'

In [ ]:
epochs = 10


from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from keras.callbacks import TensorBoard
import os.path
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions
import numpy as np
import pandas as pd



# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# dimensions of our images.
#Inception input size
img_width, img_height = 299, 299

top_layers_checkpoint_path = 'cp.top.best.hdf5'
fine_tuned_checkpoint_path = 'cp.fine_tuned.best.hdf5'
new_extended_inception_weights = 'final_weights.hdf5'

train_data_dir = '/content/train/train'
TEST_DIR = '/content/val/val'


nb_train_samples = 3138
nb_validation_samples = 800

top_epochs = 100
fit_epochs = 100

batch_size = 24

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- we have 10 classes
predictions = Dense(10, activation='softmax')(x)

# this is the model we will train
model = Model(input=base_model.input, output=predictions)

if os.path.exists(top_layers_checkpoint_path):
	model.load_weights(top_layers_checkpoint_path)
	print ("Checkpoint '" + top_layers_checkpoint_path + "' loaded.")

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'], )

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical')


#Save the model after every epoch.
mc_top = ModelCheckpoint(top_layers_checkpoint_path, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)

#Save the TensorBoard logs.
tb = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)

# train the model on the new data for a few epochs
#model.fit_generator(...)



if (os.path.exists('model1.tfl.meta')):
    model.load('./model1.tfl')
else:
    model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=top_epochs,
    validation_data=0,
    nb_val_samples=nb_validation_samples // batch_size,
    callbacks=[mc_top, tb])
    model.save('mode1.tfl')


# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)


#Save the model after every epoch.
mc_fit = ModelCheckpoint(fine_tuned_checkpoint_path, monitor='val_acc', verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)


if os.path.exists(fine_tuned_checkpoint_path):
	model.load_weights(fine_tuned_checkpoint_path)
	print ("Checkpoint '" + fine_tuned_checkpoint_path + "' loaded.")

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 172 layers and unfreeze the rest:
for layer in model.layers[:172]:
   layer.trainable = False
for layer in model.layers[172:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
#model.fit_generator(...)

if (os.path.exists('model2.tfl.meta')):
    model.load('./model2.tfl')
else:
    model.fit_generator(
    train_generator,
    samples_per_epoch=nb_train_samples // batch_size,
    nb_epoch=fit_epochs,
    validation_data=0,
    nb_val_samples=nb_validation_samples // batch_size,
    callbacks=[mc_fit, tb])    
    model.save('mode2.tfl')



def preprocess_input_inc(x):
    x /= 255.
    x -= 0.5
    x *= 2.
    return x

testing_pre = []

for imagep in os.listdir(TEST_DIR):
    image_path = os.path.join(TEST_DIR, imagep)
    img = image.load_img(image_path, target_size=(299, 299))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis = 0)
    x = preprocess_input_inc(x)
    preds = model.predict(x)
    print(imagep, preds, np.argmax(preds) + 1)
    testing_pre.append([imagep,np.argmax(preds) + 1])

Csv = pd.DataFrame(testing_pre)
Csv.to_csv("subm1.csv")


print('end')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:52: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:102: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=0, callbacks=[<keras.ca..., steps_per_epoch=5, epochs=100, validation_steps=33)`


Found 3138 images belonging to 10 classes.
Epoch 1/100
5/5 [==============================] - 37s 7s/step - loss: 8.5101 - acc: 0.1833
Epoch 2/100


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


5/5 [==============================] - 27s 5s/step - loss: 11.4013 - acc: 0.1522
Epoch 3/100
5/5 [==============================] - 28s 6s/step - loss: 9.4189 - acc: 0.2583
Epoch 4/100
5/5 [==============================] - 28s 6s/step - loss: 9.7618 - acc: 0.2167
Epoch 5/100
5/5 [==============================] - 28s 6s/step - loss: 8.5908 - acc: 0.2250
Epoch 6/100
5/5 [==============================] - 28s 6s/step - loss: 7.6517 - acc: 0.3833
Epoch 7/100
5/5 [==============================] - 28s 6s/step - loss: 8.6183 - acc: 0.2000
Epoch 8/100
5/5 [==============================] - 29s 6s/step - loss: 8.8016 - acc: 0.2333
Epoch 9/100
5/5 [==============================] - 28s 6s/step - loss: 7.4933 - acc: 0.2667
Epoch 10/100
5/5 [==============================] - 28s 6s/step - loss: 6.8836 - acc: 0.4250
Epoch 11/100
5/5 [==============================] - 28s 6s/step - loss: 8.1353 - acc: 0.3333
Epoch 12/100
5/5 [==============================] - 28s 6s/step - loss: 5.8773 - acc: 0.4

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:149: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:149: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., validation_data=0, callbacks=[<keras.ca..., steps_per_epoch=5, epochs=100, validation_steps=33)`


Epoch 1/100
5/5 [==============================] - 60s 12s/step - loss: 0.9451 - acc: 0.7000
Epoch 2/100
5/5 [==============================] - 49s 10s/step - loss: 0.4694 - acc: 0.8250
Epoch 3/100
5/5 [==============================] - 50s 10s/step - loss: 0.5201 - acc: 0.8167
Epoch 4/100
5/5 [==============================] - 49s 10s/step - loss: 0.4937 - acc: 0.8500
Epoch 5/100
5/5 [==============================] - 49s 10s/step - loss: 0.4689 - acc: 0.8750
Epoch 6/100
5/5 [==============================] - 50s 10s/step - loss: 0.3212 - acc: 0.8833
Epoch 7/100
5/5 [==============================] - 49s 10s/step - loss: 0.2553 - acc: 0.8833
Epoch 8/100
5/5 [==============================] - 50s 10s/step - loss: 0.3826 - acc: 0.8833
Epoch 9/100
5/5 [==============================] - 49s 10s/step - loss: 0.4433 - acc: 0.8417
Epoch 10/100
5/5 [==============================] - 50s 10s/step - loss: 0.3411 - acc: 0.9000
Epoch 11/100
5/5 [==============================] - 50s 10s/step - lo